In [1]:
import pandas as pd
import os
import collections 
import spacy
import pickle
from spacy.tokenizer import Tokenizer
import numpy as np
import pickle




nlp = spacy.load('en')
tokenizer = Tokenizer(nlp.vocab)



filename = "full_articles.csv"
filepath = os.path.join(os.getcwd(), filename)
data = pd.read_csv(filepath)

data.dropna(how="any", subset=["title", "content", "publication"], inplace=True)

SEQ_LENGTH = 30
STEP = 1
SAVE_DIR = "pickles"

In [42]:
titles = data.content[:100].tolist()
contents = data.content[:100].tolist() # change to all later


titles = [title.lower() for title in titles] # change to all later
contents = [content.lower() for content in contents ] # change to all later

content_words = [word.text for doc in tokenizer.pipe(contents, batch_size=128) for word in doc]
title_words = [word.text for doc in tokenizer.pipe(titles, batch_size=128) for word in doc]

In [43]:
content_count = collections.Counter(content_words)
title_count = collections.Counter(title_words)

content_vocab = list(sorted([item[0] for item in content_count.most_common()]))
title_vocab = list(sorted([item[0] for item in title_count.most_common()]))

content_word_dict = [{x: i} for i,x in enumerate(content_vocab)]
title_word_dict = [{x: i} for i,x in enumerate(title_vocab)]



In [53]:
name = "content"
PKL_SAVE = f"{name}.pkl"
with open(os.path.join("pickles",PKL_SAVE), "wb") as pkl_file:
    pickle.dump((content_words, content_vocab, content_word_dict), pkl_file)
    
name = "title"
PKL_SAVE = f"{name}.pkl"
with open(os.path.join("pickles",PKL_SAVE), "wb") as pkl_file:
    pickle.dump((content_words, content_vocab, content_word_dict), pkl_file)

In [56]:
SEQ_LENGTH = 30
STEP_SIZE = 1

X_raw_content = []    
y_raw_content = []

for i in range(0, len(content_words) - SEQ_LENGTH, STEP_SIZE):
    sequence = content_words[i:i+SEQ_LENGTH]
    next_word = content_words[i+SEQ_LENGTH]
    X_raw_content.append(sequence)
    y_raw_content.append(next_word)



In [57]:
# ONLY DOING content RN
# X = (num_sequences, SEQ_LENGTH, len(vocab))
# Y = (num_sequences, len_vocab)
n_sequences = len(X_raw_content)
n_vocab = len(content_vocab)
X_content = np.zeros(shape=(n_sequences, SEQ_LENGTH, n_vocab))
y_content = np.zeros(shape=(n_sequences, n_vocab))

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Bidirectional, LSTM, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
n_nodes = 258
model = Sequential()
model.add(Bidirectional(LSTM(n_nodes, activation="relu"), input_shape = (SEQ_LENGTH, n_vocab)))
model.add(Dropout(0.6))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
optimizer = Adam(lr=0.001)
callbacks = [EarlyStopping(patience=2, monitor="val_loss")]
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=[categorical_accuracy])
MODEL_CHECK_DIR = "/checkpoints"
callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=MODEL_CHECK_DIR + "/" + 'model_gen.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, mode='auto', period=2)]

batch_size = 32
epochs = 50

model.fit(X_content, y_content,
             batch_size = batch_size,
             epochs=epochs,
             shuffle=True,
              validation_split=0.1
             )

md.save(MODEL_CHECK_DIR + "/" + 'model_gen.h5')